In [ ]:
import matplotlib.pyplot as plt
import MDAnalysis as mda
import numpy as np
from matplotlib.markers import MarkerStyle as markerstyle
import math
from scipy.interpolate import griddata
import pandas
from scipy import constants
import matplotlib.patches as patches
import matplotlib.cm as cm

In [ ]:
"""
------- Defining Constants ------------
"""
k = constants.value('Boltzmann constant')
Ava_no = constants.value('Avogadro constant')
temp = 310
kbt = (k*temp*Ava_no)

## Read Solution data

In [ ]:
#Energies_File
#Energies_File
ener_file = "ADD PATH TO REWEIGHTED ENERGIES FILE HERE"  #Reweighted Energy file For e.g. - "Reweighted_Energies.dat"
dist_file = "ADD PATH TO MASTER DATA FILE"  # Distance file For e.g. - "Final_data.txt"

# ener_col = ['Timestep','Bond','G96','Imp','LJ','Col','Pot','Volume','Col-Pro-Pro','LJ-Pro-Pro','Col-Pro-W','LJ-Pro-W','Col-W-W','LJ-W-W']
ener_data = pandas.read_csv(ener_file,comment='#',sep='\s+',dtype=np.float64)   #Reweighted Energy file
dist_data = pandas.read_csv(dist_file,comment='#',sep='\t',dtype=np.float64)   #Distance file


#Remove duplicates 
dist_data = dist_data.drop_duplicates(subset=['Timestep'],ignore_index=True)
ener_data = ener_data.drop_duplicates(subset=['Timestep'],ignore_index=True)



# #Merge two dataframes
master_data3D = dist_data.join(ener_data.set_index('Timestep'),on='Timestep',how='inner',lsuffix = '_mda',rsuffix = '_plumed')

### Filter and classify bound vs unbound configs

In [ ]:
bound_mask = (master_data3D['d1_mda'] < 13.0) & (master_data3D['d2_mda'] < 13.0)
# bound_mask = (master_data['RMSD-BtoA'] < 2.0)
unbound_mask = ((master_data3D['d1_mda'] > 13.0) | (master_data3D['d2_mda'] > 13.0))

bd_data3D = master_data3D[bound_mask]
unb_data3D = master_data3D[unbound_mask]

## Read Membrane data

In [ ]:

#Energies_File
ener_file = "ADD PATH TO REWEIGHTED ENERGIES FILE HERE"  #Reweighted Energy file For e.g. - "Reweighted_Energies.dat"
dist_file = "ADD PATH TO MASTER DATA FILE"  # Distance file For e.g. - "Final_data.txt"

# ener_col = ['Timestep','Bond','G96','ImPDih','LJ','Col','Pot','Col-Mem-Mem','LJ-Mem-Mem','Col-Mem-W','LJ-Mem-W','Col-Pro-Mem','LJ-Pro-Mem','Col-W-W','LJ-W-W','Col-Pro-W','LJ-Pro-W','Col-Pro-Pro','LJ-Pro-Pro']

ener_data = pandas.read_csv(ener_file,comment='#',sep='\s+',dtype=np.float64)   #Reweighted Energy file
dist_data = pandas.read_csv(dist_file,comment='#',sep='\t',dtype=np.float64)   #Distance file


#Remove duplicates 
dist_data = dist_data.drop_duplicates(subset=['Timestep'],ignore_index=True)
ener_data = ener_data.drop_duplicates(subset=['Timestep'],ignore_index=True)

mask1 = dist_data['Timestep']%1000 == 0
dist_data02 = dist_data[mask1]
dist_data02.reset_index(drop=True,inplace=True)
mask1 = ener_data['Timestep']<= 220000000
ener_data02 = ener_data[mask1]
ener_data02.reset_index(drop=True,inplace=True)

# #Merge two dataframes
master_data2D = dist_data02.join(ener_data02.set_index('Timestep'),on='Timestep',how='inner',lsuffix = '_mda',rsuffix = '_plumed')

In [ ]:
bound_mask = (master_data2D['d1_mda'] < 13.0) & (master_data2D['d2_mda'] < 13.0)
# bound_mask = (master_data['RMSD-BtoA'] < 2.0)
unbound_mask = ((master_data2D['d1_mda'] > 13.0) | (master_data2D['d2_mda'] > 13.0))

bd_data2D = master_data2D[bound_mask]
unb_data2D = master_data2D[unbound_mask]

In [ ]:
rwt_bool = False
if rwt_bool:
    bd_rwt3D = bd_data3D['rbias']
    unb_rwt3D = unb_data3D['rbias']

    bd_rwt2D = bd_data2D['rbias']
    unb_rwt2D = unb_data2D['rbias']
else:
    bd_rwt3D = np.ones(len(bd_data3D))
    unb_rwt3D = np.ones(len(unb_data3D))

    bd_rwt2D = np.ones(len(bd_data2D))
    unb_rwt2D = np.ones(len(unb_data2D))

#### Plotting Parameters

In [ ]:
f_size=15
leg_size=16
tick_size=14
fig_size= (6,8)


In [ ]:
fig,[ax,ax2d] = plt.subplots(2,1,figsize=fig_size,sharex=True)
col_terms = 'Col-Pro-Pro'

ax.hist(bd_data3D[col_terms],bins=50,alpha=0.8,weights=bd_rwt3D,label='Bound',color='steelblue',density=True)
ax.hist(unb_data3D[col_terms],bins=50,alpha=0.8,weights=unb_rwt3D,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.set_title('3D',fontsize=f_size)
ax.set_xlim(-1400,-850)

ax2d.hist(bd_data2D[col_terms],bins=50,alpha=0.8,weights=bd_rwt2D,label='Bound',color='steelblue',density=True)
ax2d.hist(unb_data2D[col_terms],bins=50,alpha=0.8,weights=unb_rwt2D,label='Unbound',color='gold',density=True)
ax2d.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax2d.tick_params(axis='both',labelsize=tick_size)
ax2d.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax2d.set_ylabel('Frequency',fontsize=f_size)
ax2d.set_title('2D',fontsize=f_size)
ax2d.set_xlim(-1400,-850)
fig.suptitle("Col: Pro-Pro Energy",fontsize=f_size)
fig.tight_layout()

fig,[ax,ax2d] = plt.subplots(2,1,figsize=fig_size,sharex=True)
col_terms = 'Col-Pro-W'

ax.hist(bd_data3D[col_terms],bins=50,alpha=0.8,weights=bd_rwt3D,label='Bound',color='steelblue',density=True)
ax.hist(unb_data3D[col_terms],bins=50,alpha=0.8,weights=unb_rwt3D,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.set_title('3D',fontsize=f_size)

ax2d.hist(bd_data2D[col_terms],bins=50,alpha=0.8,weights=bd_rwt2D,label='Bound',color='steelblue',density=True)
ax2d.hist(unb_data2D[col_terms],bins=50,alpha=0.8,weights=unb_rwt2D,label='Unbound',color='gold',density=True)
ax2d.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax2d.tick_params(axis='both',labelsize=tick_size)
ax2d.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax2d.set_ylabel('Frequency',fontsize=f_size)
ax2d.set_title('2D',fontsize=f_size)
fig.suptitle("Col: Pro-W Energy",fontsize=f_size)
fig.tight_layout()

fig,ax2d = plt.subplots(figsize=(6,4))
col_terms = 'Col-Pro-Mem'


ax2d.hist(bd_data2D[col_terms],bins=50,alpha=0.8,weights=bd_rwt2D,label='Bound',color='steelblue',density=True)
ax2d.hist(unb_data2D[col_terms],bins=50,alpha=0.8,weights=unb_rwt2D,label='Unbound',color='gold',density=True)
ax2d.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax2d.tick_params(axis='both',labelsize=tick_size)
ax2d.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax2d.set_ylabel('Frequency',fontsize=f_size)
ax2d.set_title('2D',fontsize=f_size)
fig.suptitle("Col: Pro-Mem Energy",fontsize=f_size)
fig.tight_layout()
plt.show()



In [ ]:
fig,[ax,ax2d] = plt.subplots(2,1,figsize=fig_size,sharex=True)
col_terms = 'LJ-Pro-Pro'

ax.hist(bd_data3D[col_terms],bins=50,alpha=0.8,weights=bd_rwt3D,label='Bound',color='steelblue',density=True)
ax.hist(unb_data3D[col_terms],bins=50,alpha=0.8,weights=unb_rwt3D,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
# ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.set_title('3D',fontsize=f_size)
# ax.set_xlim(-1400,-850)

ax2d.hist(bd_data2D[col_terms],bins=50,alpha=0.8,weights=bd_rwt2D,label='Bound',color='steelblue',density=True)
ax2d.hist(unb_data2D[col_terms],bins=50,alpha=0.8,weights=unb_rwt2D,label='Unbound',color='gold',density=True)
ax2d.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax2d.tick_params(axis='both',labelsize=tick_size)
ax2d.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax2d.set_ylabel('Frequency',fontsize=f_size)
ax2d.set_title('2D',fontsize=f_size)
# ax2d.set_xlim(-1400,-850)
fig.suptitle("LJ: Pro-Pro Energy",fontsize=f_size)
fig.tight_layout()

fig,[ax,ax2d] = plt.subplots(2,1,figsize=fig_size,sharex=True)
col_terms = 'LJ-Pro-W'

ax.hist(bd_data3D[col_terms],bins=50,alpha=0.8,weights=bd_rwt3D,label='Bound',color='steelblue',density=True)
ax.hist(unb_data3D[col_terms],bins=50,alpha=0.8,weights=unb_rwt3D,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
# ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.set_title('3D',fontsize=f_size)

ax2d.hist(bd_data2D[col_terms],bins=50,alpha=0.8,weights=bd_rwt2D,label='Bound',color='steelblue',density=True)
ax2d.hist(unb_data2D[col_terms],bins=50,alpha=0.8,weights=unb_rwt2D,label='Unbound',color='gold',density=True)
ax2d.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax2d.tick_params(axis='both',labelsize=tick_size)
ax2d.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax2d.set_ylabel('Frequency',fontsize=f_size)
ax2d.set_title('2D',fontsize=f_size)
fig.suptitle("LJ: Pro-W Energy",fontsize=f_size)
fig.tight_layout()

fig,ax2d = plt.subplots(figsize=(6,4))
col_terms = 'LJ-Pro-Mem'


ax2d.hist(bd_data2D[col_terms],bins=50,alpha=0.8,weights=bd_rwt2D,label='Bound',color='steelblue',density=True)
ax2d.hist(unb_data2D[col_terms],bins=50,alpha=0.8,weights=unb_rwt2D,label='Unbound',color='gold',density=True)
ax2d.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax2d.tick_params(axis='both',labelsize=tick_size)
ax2d.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax2d.set_ylabel('Frequency',fontsize=f_size)
ax2d.set_title('2D',fontsize=f_size)
fig.suptitle("LJ: Pro-Mem Energy",fontsize=f_size)
fig.tight_layout()
plt.show()



In [ ]:
def plot_vs_cutoff(plot_data,ener_col,cutoffs,ax,colors=None):
    n_configs=[]
    prev_cutoff = 0.0
    for cutoff in cutoffs:
        mask = (plot_data['RMSD-BtoA'] < cutoff) & (plot_data['RMSD-BtoA'] >= prev_cutoff)
        data = plot_data[mask]
        energy = data[[ener_col]].sum(axis=1)
        if colors is not None:
            ax.hist(energy,bins=50,alpha=0.8,weights=np.ones(len(energy)),label=f'{cutoff} nm',density=True,color=colors[cutoff])
        else:
            ax.hist(energy,bins=50,alpha=0.8,weights=np.ones(len(energy)),label=f'{cutoff} nm',density=True)

        n_configs.append(len(energy))
        prev_cutoff = cutoff

    return n_configs


def plot_vs_cutoffMem(plot_data,ener_col,cutoffs,ax,colors=None):
    n_configs=[]
    prev_cutoff = 0.0
    for cutoff in cutoffs:
        mask = (plot_data['RMSD-B'] < cutoff) & (plot_data['RMSD-B'] >= prev_cutoff)
        data = plot_data[mask]
        energy = data[ener_col]
        if colors is not None:
            ax.hist(energy,bins=50,alpha=0.8,weights=np.ones(len(energy)),label=f'{cutoff} nm',density=True,color=colors[cutoff])
        else:
            # print(energy)
            ax.hist(energy,bins=50,alpha=0.8,weights=np.ones(len(energy)),label=f'{cutoff} nm',density=True)

        n_configs.append(len(energy))
        prev_cutoff = cutoff

    return n_configs



In [ ]:
sorted_3D = master_data3D.sort_values(by='RMSD-BtoA',ascending=True)
sorted_2D = master_data2D.sort_values(by='RMSD-B',ascending=True)
# sorted_pseudo = master_data_pseudo.sort_values(by='RMSD-B',ascending=True)

nbins=50

hist3D = np.histogram(sorted_3D['RMSD-BtoA'],bins=nbins)
hist2D = np.histogram(sorted_2D['RMSD-B'],bins=nbins)


rmsd_bins3D = hist3D[1]
rmsd_bins2D = hist2D[1]




energies_3D= {}
energies_3D_rwt = {}
energies_2D= {}
energies_2D_rwt = {}
energies_pseudo= {}
energies_pseudo_rwt = {}

energies_3D_std = {}
energies_2D_std = {}
col_names = 'Col-Pro-Pro','LJ-Pro-Pro','Col-Pro-W','LJ-Pro-W'

for i in range(nbins):
    mask3D = (sorted_3D['RMSD-BtoA'] > rmsd_bins3D[i]) & (sorted_3D['RMSD-BtoA'] < rmsd_bins3D[i+1])
    mask2D = (sorted_2D['RMSD-B'] > rmsd_bins2D[i]) & (sorted_2D['RMSD-B'] < rmsd_bins2D[i+1])
    
    for col in col_names:
        if col not in energies_3D:
            energies_3D[col] = []
            energies_3D_rwt[col] = []
            energies_2D[col] = []
            energies_2D_rwt[col] = []
            energies_pseudo[col] = []
            energies_pseudo_rwt[col] = []

            energies_3D_std[col] = []
            energies_2D_std[col] = []

        energies_3D[col].append(np.average(sorted_3D[mask3D][col]))
        energies_3D_rwt[col].append(np.average(sorted_3D[mask3D][col],weights=np.exp(sorted_3D[mask3D]['rbias']/kbt)))
        energies_3D_std[col].append(np.std(sorted_3D[mask3D][col]))

        energies_2D[col].append(np.average(sorted_2D[mask2D][col]))
        energies_2D_rwt[col].append(np.average(sorted_2D[mask2D][col],weights=np.exp(sorted_2D[mask2D]['rbias']/kbt)))
        energies_2D_std[col].append(np.std(sorted_2D[mask2D][col]))

        


In [ ]:
"""
Plot Reweighted energies
"""
fig,ax=plt.subplots(2,2,figsize=(10,8))

t_size= 14
lw = 3
f_size= 16
for i in range(len(col_names)):
    row_idx = i//2
    col_idx = i%2

   

    unb_mask = (rmsd_bins3D[:-1] > 12.0) & (rmsd_bins3D[:-1] < 20.0)
    zero_point = np.mean(np.array(energies_3D_rwt[col_names[i]])[unb_mask])
    energies_3D_rwt[col_names[i]] = np.array(energies_3D_rwt[col_names[i]]) - zero_point

    unb_mask = (rmsd_bins2D[:-1] > 12.0) & (rmsd_bins2D[:-1] < 20.0)
    zero_point = np.mean(np.array(energies_2D_rwt[col_names[i]])[unb_mask])
    energies_2D_rwt[col_names[i]] = np.array(energies_2D_rwt[col_names[i]]) - zero_point

    low_lim = np.min([np.min(energies_3D_rwt[col_names[i]]),np.min(energies_2D_rwt[col_names[i]])])
    upp_lim = np.max([np.max(energies_3D_rwt[col_names[i]]),np.max(energies_2D_rwt[col_names[i]])])

    ax[row_idx,col_idx].plot(rmsd_bins3D[:-1],energies_3D_rwt[col_names[i]],label='3D',color='navy',lw=lw,alpha=0.8)
    ax[row_idx,col_idx].plot(rmsd_bins2D[:-1],energies_2D_rwt[col_names[i]],label='2D',color='firebrick',lw=lw,alpha=0.8)
    # ax[row_idx,col_idx].plot(rmsd_bins_pseudo[:-1],energies_pseudo[col_names[i]],label='Pseudo',color='teal',lw=lw,alpha=0.8)

    ax[row_idx,col_idx].vlines(2.0,low_lim,upp_lim,color='black',linestyle='--',lw=3)

    ax[row_idx,col_idx].set_title(col_names[i],fontsize=f_size+4)

ax[1,1].legend(fontsize=f_size)

for ax_ in ax.flat:
    ax_.set_xlabel('RMSD (nm)',fontsize=f_size)
    ax_.set_ylabel('Energy (kJ/mol)',fontsize=f_size)
    
    ax_.tick_params(axis='both',labelsize=t_size)
    
    ax_.set_xlim(0,19.0)
fig.tight_layout()